In [ ]:
# Configuring the path of kaggle file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

 97% 786M/812M [00:17<00:00, 40.3MB/s]
100% 812M/812M [00:17<00:00, 48.2MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [ ]:
dir = "/content/dogscats/train"
categories = ['cats',"dogs"]

In [ ]:
for i in categories:
  folder = os.path.join(dir,i)
  print(folder)

/content/dogscats/train/cats
/content/dogscats/train/dogs


In [ ]:
# for i in categories:
#   folder = os.path.join(dir,i)
#   for img in os.listdir(folder):
#     img_path = os.path.join(folder,img)
#     print(img_path)
#     break

In [ ]:
# for i in categories:
#   folder = os.path.join(dir,i)
#   for img in os.listdir(folder):
#     img_path = os.path.join(folder,img)
#     img_arr = cv2.imread(img_path)
#     plt.imshow(img_arr)
#     break

In [ ]:
# for i in categories:
#   folder = os.path.join(dir,i)
#   for img in os.listdir(folder):
#     img_path = os.path.join(folder,img)
#     img_arr = cv2.imread(img_path)
#     img_arr =cv2.resize(img_arr,(100,100))
#     plt.imshow(img_arr)
#     break

In [ ]:
data = []
for i in categories:
  folder = os.path.join(dir,i)
  label = categories.index(i)
  for img in os.listdir(folder):
    img_path = os.path.join(folder,img)
    img_arr = cv2.imread(img_path)
    img_arr =cv2.resize(img_arr,(100,100))
    data.append([img_arr,label])

In [ ]:
len(data)

23000

In [ ]:
random.shuffle(data)

In [ ]:
data[0]

[array([[[118, 176, 152],
         [102, 170, 145],
         [ 94, 184, 155],
         ...,
         [229, 253, 251],
         [168, 207, 192],
         [113, 188, 160]],
 
        [[164, 205, 184],
         [150, 199, 177],
         [112, 185, 159],
         ...,
         [252, 250, 255],
         [171, 206, 195],
         [ 77, 173, 143]],
 
        [[167, 204, 184],
         [161, 207, 185],
         [118, 186, 161],
         ...,
         [209, 233, 239],
         [163, 196, 189],
         [135, 200, 179]],
 
        ...,
 
        [[  0,  36,   4],
         [  8,  28,   5],
         [  8,  40,   9],
         ...,
         [ 23,  91,  50],
         [ 12,  91,  64],
         [ 19, 111,  64]],
 
        [[  0,  33,   1],
         [  5,  31,   7],
         [  0,  38,   6],
         ...,
         [  4,  62,  27],
         [ 48, 114,  95],
         [  0,  76,  34]],
 
        [[ 50, 101,  67],
         [  0,  32,   6],
         [  0,  33,   2],
         ...,
         [  0,  46,   3],
  

In [ ]:
X = []
y = []

In [ ]:
for features,label in data:
  X.append(features)
  y.append(label)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
len(X),len(y)

(23000, 23000)

In [ ]:
X[0:2]

array([[[[118, 176, 152],
         [102, 170, 145],
         [ 94, 184, 155],
         ...,
         [229, 253, 251],
         [168, 207, 192],
         [113, 188, 160]],

        [[164, 205, 184],
         [150, 199, 177],
         [112, 185, 159],
         ...,
         [252, 250, 255],
         [171, 206, 195],
         [ 77, 173, 143]],

        [[167, 204, 184],
         [161, 207, 185],
         [118, 186, 161],
         ...,
         [209, 233, 239],
         [163, 196, 189],
         [135, 200, 179]],

        ...,

        [[  0,  36,   4],
         [  8,  28,   5],
         [  8,  40,   9],
         ...,
         [ 23,  91,  50],
         [ 12,  91,  64],
         [ 19, 111,  64]],

        [[  0,  33,   1],
         [  5,  31,   7],
         [  0,  38,   6],
         ...,
         [  4,  62,  27],
         [ 48, 114,  95],
         [  0,  76,  34]],

        [[ 50, 101,  67],
         [  0,  32,   6],
         [  0,  33,   2],
         ...,
         [  0,  46,   3],
        

In [ ]:
y[0:2]

array([0, 1])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
X_train = X_train/255
X_test = X_test/255

In [ ]:
X_train.shape,X_test.shape

((18400, 100, 100, 3), (4600, 100, 100, 3))

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg16 = VGG16()

In [ ]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
type(vgg16)

keras.engine.functional.Functional

In [ ]:
for layer in vgg16.layers:
  print(layer)

In [ ]:
for layer in vgg16.layers[:-1]:
  print(layer)

In [ ]:
model = Sequential()

In [ ]:
for layer in vgg16.layers[:-1]:
  model.add(layer)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit(X_train,y_train,epochs=5,batch_size=128)

#### Dog

In [ ]:
input_img = cv2.imread("/content/dog.jpg")

In [ ]:
input_img.shape

(1200, 1600, 3)

In [ ]:
input2 = cv2.resize(input_img,(100,100))

In [ ]:
input2 = input2.reshape(1,100,100,3)

In [ ]:
model.predict(input2)

1/1 [==============================] - 0s 94ms/step


array([[1.]], dtype=float32)